In [14]:
import os
import glob
import re
import pandas as pd
import plotly
import plotly.graph_objs as go
from IPython.display import display, HTML
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.__version__

'3.7.0'

In [15]:
init_notebook_mode(connected=True)
pd.set_option('display.precision', 1)

# DATA PREPARATION

Read data from excel file into a pandas data frame.

In [16]:
def extract_xlsx(filename):
    fsiDataFrame = pd.read_excel(filename)
    score = fsiDataFrame['Total']
    countries = fsiDataFrame['Country']
    return fsiDataFrame, score, countries

Read data from csv file and prepare ISO - 3 Country code dictonary.
- Type: DataFrame
- String form:

                                COUNTRY CODE
           0               Afghanistan  AFG
           1                   Albania  AL <...> ia  ZMB
           223                Zimbabwe  ZWE
           224                Eswatini  SWZ
           
           [225 rows x 2 columns]
Length:      225

In [17]:
def iso3_code():
    isoDataFrame = pd.read_csv('./data/ISO-3166-1-alpha-3.csv')
    isoCodeDict = dict(zip(isoDataFrame['COUNTRY'], isoDataFrame['CODE']))
    return isoCodeDict

# PREPARE DATAFAME FOR CHOROPLETH 

In [18]:
def country_code_mapping(file):
    _,score,countries = extract_xlsx(file)
    codeDict = iso3_code()
    codeList = list()
    for countryName in countries:
        codeList.append(codeDict[countryName.strip(' ')])
    return score, countries,codeList

In [19]:
def combine_data(file):  
    score, countries, code = country_code_mapping(file=file)
    data = dict(ISOCode = code, FsScore = score, Country = countries)
    choroplethData = pd.DataFrame.from_dict(data)
    return choroplethData['ISOCode'], choroplethData['FsScore'], choroplethData['Country']

# PREPARING DATA AND FRAMES

##### COLORSCALE FOR COLORBAR
        [
        [0, 'rgb(25,25,112)'], #'MidnightBlue', 
        [0.08333333, 'rgb(65,105,225)'], #'RoyalBlue',
        [0.16666667, 'rgb(100,149,237)'],#'CornflowerBlue', 
        [0.33333333, 'rgb(0,100,0)'],#'DarkGreen',
        [0.41666667, 'rgb(0,128,0)'],#'Green',
        [0.5, 'rgb(50,205,50)'],#'LimeGreen',
        [0.58333333, 'rgb(255,255,0)'],#'Yellow',
        [0.66666667, 'rgb(255,215,0)'],#'Gold',
        [0.75, 'rgb(255,140,0)'],#'DarkOrange',
        [0.83333333, 'rgb(255,69,0)'],#'OrangeRed',
        [0.91666667, 'rgb(178,34,34)'],#'FireBrick', 
        [1, 'rgb(139,0,0)'] # 'DarkRed'
      ]


In [20]:
def make_frame(file):
    c,s,c_name = combine_data(file=file)
    frameData = go.Choropleth(   
                    locations = c,
                    z = s,
                    text = c_name,
                    colorscale =  [
                                    [0, 'rgb(25,25,112)'],
                                    [0.0833, 'rgb(25,25,112)'], # 0 - 10

                                    [0.0833, 'rgb(65,105,225)'], 
                                    [0.1666, 'rgb(65,105,225)'], # 11 - 20

                                    [0.1666, 'rgb(100,149,237)'], # 21 - 30
                                    [0.25, 'rgb(100,149,237)'],

                                    [0.25, 'rgb(0,100,0)'], # 31 - 40
                                    [0.3333, 'rgb(0,100,0)'],

                                    [0.3333, 'rgb(0,128,0)'], #41 - 50
                                    [0.4166, 'rgb(0,128,0)'],

                                    [0.4166, 'rgb(50,205,50)'], #51 - 60
                                    [0.5, 'rgb(50,205,50)'],

                                    [0.5, 'rgb(255,255,0)'], #61 - 70
                                    [0.5833, 'rgb(255,255,0)'],

                                    [0.5833, 'rgb(255,215,0)'], #71 - 80
                                    [0.6666, 'rgb(255,215,0)'],

                                    [0.6666, 'rgb(255,140,0)'], #81 - 90
                                    [0.75, 'rgb(255,140,0)'],

                                    [0.75, 'rgb(255,69,0)'], #91 - 100
                                    [0.8333, 'rgb(255,69,0)'],

                                    [0.8333, 'rgb(178,34,34)'], #101 - 110
                                    [0.9166, 'rgb(178,34,34)'],

                                    [0.9166, 'rgb(139,0,0)'], #111 - 120
                                    [1, 'rgb(139,0,0)']
                                    ],
                    zmin = 0.0,
                    zmax = 120.0,
                    autocolorscale = False,
                    reversescale = False,
                    marker = go.choropleth.Marker(
                    line = go.choropleth.marker.Line(
                            color = 'rgb(105,105,105)',
                            width = 0.25)),
                    colorbar = go.choropleth.ColorBar(
                            title = {'text': 'FSI Score', 'font':{'size': 20}},
                            tickmode = 'array',
                            tickvals = list(range(0,130,10)),
                            ticktext = ['0','','Sustainable','','', 'Stable','', '', 'Warning','', '', 'Alert']))
    return frameData

In [21]:
def make_data(file = './data/xlsx/fsi-2006.xlsx'):
    c,s,c_name = combine_data(file=file)
    frameData = [go.Choropleth(   
                    locations = c,
                    z = s,
                    text = c_name,
                    colorscale =  [
                                    [0, 'rgb(25,25,112)'],
                                    [0.0833, 'rgb(25,25,112)'], # 0 - 10

                                    [0.0833, 'rgb(65,105,225)'], 
                                    [0.1666, 'rgb(65,105,225)'], # 11 - 20

                                    [0.1666, 'rgb(100,149,237)'], # 21 - 30
                                    [0.25, 'rgb(100,149,237)'],

                                    [0.25, 'rgb(0,100,0)'], # 31 - 40
                                    [0.3333, 'rgb(0,100,0)'],

                                    [0.3333, 'rgb(0,128,0)'], #41 - 50
                                    [0.4166, 'rgb(0,128,0)'],

                                    [0.4166, 'rgb(50,205,50)'], #51 - 60
                                    [0.5, 'rgb(50,205,50)'],

                                    [0.5, 'rgb(255,255,0)'], #61 - 70
                                    [0.5833, 'rgb(255,255,0)'],

                                    [0.5833, 'rgb(255,215,0)'], #71 - 80
                                    [0.6666, 'rgb(255,215,0)'],

                                    [0.6666, 'rgb(255,140,0)'], #81 - 90
                                    [0.75, 'rgb(255,140,0)'],

                                    [0.75, 'rgb(255,69,0)'], #91 - 100
                                    [0.8333, 'rgb(255,69,0)'],

                                    [0.8333, 'rgb(178,34,34)'], #101 - 110
                                    [0.9166, 'rgb(178,34,34)'],

                                    [0.9166, 'rgb(139,0,0)'], #111 - 120
                                    [1, 'rgb(139,0,0)']
                                    ],
                    zmin = 0.0,
                    zmax = 120.0,
                    autocolorscale = False,
                    reversescale = False,
                    marker = go.choropleth.Marker(
                    line = go.choropleth.marker.Line(
                            color = 'rgb(105,105,105)',
                            width = 0.25)),
                    colorbar = go.choropleth.ColorBar(
                            title = {'text': 'FSI Score', 'font':{'size': 20}},
                            tickmode = 'array',
                            tickvals = list(range(0,130,10)),
                            ticktext = ['0','','Sustainable','','', 'Stable','', '', 'Warning','', '', 'Alert']))]
    return frameData

# APPEND FRAMES

In [22]:
def make_frame_list():
    frames = []
    years = list()
    for file in glob.glob('./data/xlsx/*.xlsx'):
        frameList = {'data' : [], 'name' : "".join(re.findall(r'\d+', file))}
        years.append("".join(re.findall(r'\d+', file)))
        frameList['data'].append(make_frame(file=file))
        frames.append(frameList)
    return frames, years

# PREPARE STEPS FOR SLIDER

In [23]:
def make_steps(years):
    stps = list()
    for yrs in years:
        stps.append(go.layout.slider.Step(args= [[yrs], 
               {'frame':{'duration': 300, 'redraw' : False},
                'mode' : 'immediate',
             'transition' : {'duration': 300}}],
         method = 'animate',
         label = yrs,
         visible = True
                ))
    return stps

# PLOT CHOROPLETH MAP

In [28]:
def plot_choropleth():
    f, years = make_frame_list()
    data = make_data()
    layout = go.Layout(
        
    title = go.layout.Title(text = 'Fragile States Index : 2006 - 2019', 
                            font=go.layout.title.Font(size=25)),
    annotations = [{'text': '<b>Source:</b> <a href="https://fragilestatesindex.org/">The Fund For Peace</a> <br> \
    <b>Data:</b> <a href="https://fragilestatesindex.org/excel/">Fragile States Index</a>',
                    'showarrow': False,
                    'x': 0.5,
                    'xanchor':'center',
                    'y': 1,
                    'yanchor':'bottom'}],
    geo = go.layout.Geo(
    showframe = False,
    showcoastlines = False,
    showocean = False,
    oceancolor = 'rgb(173,216,230)',
    bgcolor = '#add8e6',
    projection = go.layout.geo.Projection(
        type = 'miller',           
            )
    ),
    updatemenus = [go.layout.Updatemenu(
    type='buttons',
    visible = True,
    buttons = [
        go.layout.updatemenu.Button(
            args = [None, 
            {
            'frame': {
                'duration': 500,
                'redraw': False
            },

            'fromcurrent': True, 
            'transition': {
                'duration': 300, 
                'easing': 'quadratic-in-out'
            }
            }],

            label='Play',
            visible=True, 
            method='animate'
        ),
               
         go.layout.updatemenu.Button(
            args=[[None], {
                'frame': {
                    'duration': 0,
                    'redraw': False
                },
                
            'mode': 'immediate',
            'transition': {'duration': 0}
             }],
            
            label='Pause',
            visible=True, 
            method='animate'
        )])],
    sliders = [go.layout.Slider(
            
            currentvalue = go.layout.slider.Currentvalue(
                         font = {'size': 30},
                         prefix = 'Year:',
                         visible = True,
                         xanchor = 'center'
                ),
            y = 1.005,
            yanchor = 'top',
            pad = {"t": 10, 'b': 20},
            visible = True,
            steps = make_steps(years)
    )], 
        height = 1000,
        width = 1000
                      )

    fig = dict(data = data, layout = layout, frames = f)
    iplot(fig)